In [1]:
import networkx as nx
from networkx.utils import py_random_state
from networkx.generators.random_graphs import _random_subset
from networkx.generators.classic import star_graph

import matplotlib.pyplot as plt

import scipy.stats as stats
from scipy.optimize import curve_fit

import numpy as np
import random
import time

In [2]:
def linear_search(lys, element):
    for i in range(len(lys)):
        if lys[i] == element:
            return i
    return -1

seed = np.random.RandomState()
def exploration_time (G, explorations_per_graph):
    start_node = _random_subset(G.nodes, 1, seed).pop()
        
    def exploration(node, G, explored_nodes):
        if linear_search(explored_nodes, node) < 0:
            explored_nodes += [node]
        neighbours = list(G[node])
        new_node = _random_subset(neighbours, 1, seed).pop()
        return new_node

    exploration_time = []
    explored_nodes = []
    
    for i in range(explorations_per_graph):
        t = 0
        while True:
            start_node = exploration(start_node, G, explored_nodes)

            if len(explored_nodes) == len(G.nodes):
                exploration_time += [t]
                explored_nodes.clear()
                break
            t += 1

    mean_exploration_time = sum(exploration_time)/explorations_per_graph
    print("Mean exploration time:", mean_exploration_time, "for", len(G.nodes) ,"nodes")

    return mean_exploration_time

In [3]:
time_steps = int(1e5) 
explorations_per_graph = 3

In [ ]:
t0 = time.time()

Times = []
N = [200+500*i for i in range(40)]

for i in range(len(N)):
    G = nx.complete_graph(N[i])
    Times += [exploration_time(G, explorations_per_graph)]
    
print(f"done in {int((time.time()-t0)/60)} minutes and {((time.time()-t0)%60)} seconds")

Mean exploration time: 1311.3333333333333 for 200 nodes
Mean exploration time: 4810.333333333333 for 700 nodes
Mean exploration time: 9255.0 for 1200 nodes
Mean exploration time: 12406.333333333334 for 1700 nodes
Mean exploration time: 20868.0 for 2200 nodes
Mean exploration time: 21141.333333333332 for 2700 nodes
Mean exploration time: 27996.333333333332 for 3200 nodes
Mean exploration time: 33124.333333333336 for 3700 nodes
Mean exploration time: 38340.0 for 4200 nodes
Mean exploration time: 43379.333333333336 for 4700 nodes
Mean exploration time: 43476.0 for 5200 nodes
Mean exploration time: 46642.333333333336 for 5700 nodes
Mean exploration time: 71016.66666666667 for 6200 nodes
Mean exploration time: 64101.333333333336 for 6700 nodes
Mean exploration time: 75488.66666666667 for 7200 nodes
Mean exploration time: 82226.0 for 7700 nodes
Mean exploration time: 76544.33333333333 for 8200 nodes
Mean exploration time: 86485.33333333333 for 8700 nodes
Mean exploration time: 88698.0 for 92

In [ ]:
def log_law(x, a):
    return a*x*np.log(x) 

popt, pcov = curve_fit(log_law, N, Times)

linear_fit=np.polyfit(N,Times,1)
print(linear_fit)

fit = [linear_fit[0]*n+linear_fit[1] for n in N]
fit_log = log_law(np.array(N), *popt)

fig, ax = plt.subplots()
    
ax.plot(N, Times, '.', label = f"Exploration time")
ax.plot(N, fit, '-', label = f"fit: {round(linear_fit[1],4)} + x • {round(linear_fit[0],3)}")
ax.plot(N, fit_log, '-', label = f"fit: {round(popt[0],4)} • N • log(N)")

ax.set_xlabel("Number of nodes")
ax.set_ylabel(f"Exploration time")

ax.legend()

plt.show()